In [1]:
import json
train_summary_id_to_caption_path = "/home/grads/h/hasnat.md.abdullah/open_ended_activity_analysis/zero-shot-video-to-text/experiments/summary_study_activityNet/train exp/train_summary_id_to_caption.json"
with open(train_summary_id_to_caption_path,'r') as f:
    train_summary_id_to_caption = json.load(f)

len(train_summary_id_to_caption)

705

In [3]:
import os
val_dir_path = "/home/grads/h/hasnat.md.abdullah/open_ended_activity_analysis/zero-shot-video-to-text/data/ActivityNet_captions_dataset/training"
def find_file_path(directory, filename):
    for root, dirs, files in os.walk(directory):
        if filename in files:
            return os.path.join(root, filename)

    return None

count = 0
for id in train_summary_id_to_caption.keys():
    video_id = id[2:]
    file_name = video_id+".mp4"
    file_path = find_file_path(val_dir_path, file_name)

    
    if file_path:
        # print("File found at:", file_path)
        train_summary_id_to_caption[id]["path"]=file_path
        count+=1
    # else:

    #     print(f"File not found: {id}")
    
print("FILE COUNT:",count)
with open("/home/grads/h/hasnat.md.abdullah/open_ended_activity_analysis/zero-shot-video-to-text/experiments/summary_study_activityNet/train exp/train_summary_id_to_caption.json",'w') as f:
    json.dump(train_summary_id_to_caption,f,indent=4)

FILE COUNT: 550


In [ ]:
# merging 
import json

def load_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def merge_dicts(dict1, dict2):
    merged_dict = {**dict1, **dict2}
    return merged_dict

def write_to_json_file(data, file_path):
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

if __name__ == "__main__":
    # Load data from three JSON files
    dict1 = load_json_file("/home/grads/h/hasnat.md.abdullah/open_ended_activity_analysis/zero-shot-video-to-text/experiments/summary_study_activityNet/train exp/output_train_summary_id_caption(0-353).json")
    dict2 = load_json_file("/home/grads/h/hasnat.md.abdullah/open_ended_activity_analysis/zero-shot-video-to-text/experiments/summary_study_activityNet/train exp/output_train_summary_id_caption(353-705).json")
       

    # Merge the dictionaries
    merged_dict = merge_dicts(dict1, dict2)

    # Write the merged dictionary to a new JSON file
    write_to_json_file(merged_dict, "/home/grads/h/hasnat.md.abdullah/open_ended_activity_analysis/zero-shot-video-to-text/experiments/summary_study_activityNet/val_2 exp/output_val_2_summary_id_caption.json")
    

### evaluation of train set on zero-shot captioner. 
### bleu
BLEU@1: 0.0526
BLEU@2: 0.0000
BLEU@3: 0.0000
BLEU@4: 0.0000


In [2]:
"""
evaluating the generated summaries of 50 (least freq labels) videos

"""
import os
import subprocess
import json
from tqdm import tqdm
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
import statistics
import string
def load_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data
def remove_punctuation(sentence):
    # Remove punctuations from the sentence
    no_punct = sentence.translate(str.maketrans("", "", string.punctuation))
    # Lowercase all the words
    lowercase_sentence = no_punct.lower()

    # stem words
    stemmer = nltk.stem.PorterStemmer()
    sentence = " ".join([stemmer.stem(word) for word in lowercase_sentence.split()])
    return sentence


output_val_2_summary_id_caption = load_json_file("/home/grads/h/hasnat.md.abdullah/open_ended_activity_analysis/zero-shot-video-to-text/experiments/summary_study_activityNet/train exp/output_train_summary_id_caption.json")


# bleu score calculation
reference_captions = []
candidate_captions = []
print(len(output_val_2_summary_id_caption))
for k,v in output_val_2_summary_id_caption.items():
    if v['generated_summary']!="_":
        reference_captions.append(remove_punctuation(v['summary_cap']))
        candidate_captions.append(remove_punctuation(v['generated_summary']))


# Tokenize the captions
reference_captions = [caption.split() for caption in reference_captions]
candidate_captions = [caption.split() for caption in candidate_captions]

print("len(reference_captions)",len(reference_captions))
print("len(candidate_captions)",len(candidate_captions))
# Calculate BLEU score
cumulative_bleu_scores = []
weight_list = [(1, 0, 0, 0),(0.5, 0.5, 0, 0),(0.33, 0.33, 0.33, 0),(0.25, 0.25, 0.25, 0.25)]
for n in range(1, 5):
    bleu_scores = [sentence_bleu(ref, cand, weights=weight_list[n-1]) for ref, cand in zip(reference_captions, candidate_captions)]
    
    # print(type(bleu_scores))

    average_bleu_score = statistics.mean(bleu_scores)
    cumulative_bleu_scores.append(average_bleu_score)

# Print cumulative BLEU scores
print("cumulative BLEU scores:")
for i, score in enumerate(cumulative_bleu_scores):
    print(f"BLEU@{i+1}: {score:.4f}")


705
len(reference_captions) 550
len(candidate_captions) 550


/home/grads/h/hasnat.md.abdullah/open_ended_activity_analysis/oeaa/lib/python3.6/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/grads/h/hasnat.md.abdullah/open_ended_activity_analysis/oeaa/lib/python3.6/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/grads/h/hasnat.md.abdullah/open_ended_activity_analysis/oeaa/lib/python3.6/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score

cumulative BLEU scores:
BLEU@1: 0.0526
BLEU@2: 0.0000
BLEU@3: 0.0000
BLEU@4: 0.0000


### metero for train

*METEOR Score: 0.11133880097668851

In [3]:

def remove_punctuation(sentence):
    # Remove punctuations from the sentence
    no_punct = sentence.translate(str.maketrans("", "", string.punctuation))
    # Lowercase all the words
    lowercase_sentence = no_punct.lower()

    # stem words
    stemmer = nltk.stem.PorterStemmer()
    sentence = " ".join([stemmer.stem(word) for word in lowercase_sentence.split()])
    return sentence
output_train_summary_id_caption = load_json_file("/home/grads/h/hasnat.md.abdullah/open_ended_activity_analysis/zero-shot-video-to-text/experiments/summary_study_activityNet/train exp/output_train_summary_id_caption.json")

reference_captions = []
candidate_captions = []
print(len(output_train_summary_id_caption))
for k,v in output_train_summary_id_caption.items():
    if v['generated_summary']!="_":
        reference_captions.append(remove_punctuation(v['summary_cap']))
        candidate_captions.append(remove_punctuation(v['generated_summary']))

705


In [4]:
import nltk
from nltk.translate.meteor_score import meteor_score

def calculate_meteor_score(ref_captions, cand_captions):
    scores = []
    for ref, cand in zip(ref_captions, cand_captions):
        score = meteor_score([ref], cand)
        scores.append(score)
    average_score = sum(scores) / len(scores)
    return average_score

# # Tokenize the captions
reference_captions = [caption.split() for caption in reference_captions]
candidate_captions = [caption.split() for caption in candidate_captions]

meteor_score = calculate_meteor_score(reference_captions, candidate_captions)
print("METEOR Score:", meteor_score)



METEOR Score: 0.11133880097668851
